In [ ]:
from nanopyx.liquid._le_interpolation_catmull_rom import ShiftAndMagnify as CRShiftAndMagnify
import numpy as np
from matplotlib import pyplot as plt

## Load Data

In [ ]:
from nanopyx.core.generate.noise_add_simplex import get_simplex_noise

image_vv_big = get_simplex_noise(300, 300, amplitude = 1000)
vv_large_data = np.tile(image_vv_big, (500, 1, 1))

image_v_big = get_simplex_noise(300, 300, amplitude = 1000)
v_large_data = np.tile(image_v_big, (100, 1, 1))

image_big = get_simplex_noise(300, 300, amplitude = 1000)
large_data = np.tile(image_big, (10, 1, 1))

image_medium = get_simplex_noise(100, 100, amplitude = 1000)
medium_data = np.tile(image_medium, (10, 1, 1))

image_small = get_simplex_noise(10, 10, amplitude = 1000)
small_data = np.tile(image_small, (10, 1, 1))

image_v_small = get_simplex_noise(10, 10, amplitude = 1000)
v_small_data = np.tile(image_v_small, (1, 1, 1))

print(v_small_data.shape)

In [ ]:
crsm = CRShiftAndMagnify()

In [ ]:
magnification = 5
print("Dataset shape:", v_small_data.shape)
interpolated = crsm.benchmark(v_small_data,0,0,magnification,magnification)
print(interpolated)

In [ ]:
magnification = 5
print("Dataset shape:", small_data.shape)
interpolated = crsm.benchmark(small_data,0,0,magnification,magnification)
print(interpolated)

In [ ]:
magnification = 5
print("Dataset shape:", medium_data.shape)
interpolated = crsm.benchmark(medium_data,0,0,magnification,magnification)
print(interpolated)

In [ ]:
magnification = 5
print("Dataset shape:", large_data.shape)
interpolated = crsm.benchmark(large_data,0,0,magnification,magnification)
print(interpolated)

In [ ]:
magnification = 5
print("Dataset shape:", v_large_data.shape)
interpolated = crsm.benchmark(v_large_data,0,0,magnification,magnification)
print(interpolated)

In [ ]:
magnification = 5
print("Dataset shape:", vv_large_data.shape)
interpolated = crsm.benchmark(vv_large_data,0,0,magnification,magnification)
print(interpolated)